In [1]:
import pandas as pd
import sqlite3

# Step 1: Load CSV into a DataFrame
csv_path = '/Users/vikalp.bhatnagar/Downloads/sample_large_pnc_claims_data_final.csv'
df = pd.read_csv(csv_path)

# Optional: View the first few rows to ensure it's loaded correctly
df.head()
# Step 2: Connect to SQLite database
conn = sqlite3.connect('sampledata.db')  # if the file is in the current directory
cursor = conn.cursor()
# Step 3: Insert data into the 'loss_estimation' table
df.to_sql('PNC_Claims', conn, if_exists='append', index=False)
# Step 4: Verify the insert
pd.read_sql("SELECT * FROM PNC_Claims LIMIT 5;", conn)


,ClaimNumber,Reserve_Profile,ExposureYear,IncurredLoss,UltimateLoss,AssignedEarmarkedLoss,FinancePremium,LossType,ClaimStatus,ReportedDate,SettlementDate,Currency,GeographicalLocation
0,CLM000001,Commercial,2012,9583.95,64688.85,23518.65,426.83,Gross,Closed,2016-03-14,2018-07-23,AUD,London
1,CLM000002,Workers Comp,2024,54297.95,94369.91,42134.02,3294.77,Gross,Reopened,2014-05-30,2015-03-05,CAD,Chicago
2,CLM000003,Auto Liability,2024,91409.74,109411.41,6792.00,2010.01,Gross,Reopened,2013-11-19,2015-09-19,CAD,Chicago
3,CLM000004,Commercial,2024,39409.30,102218.18,9855.28,9329.09,Net,Reopened,2014-11-22,2015-09-21,EUR,Los Angeles
4,CLM000005,Commercial,2013,83127.02,71104.61,14480.70,6760.91,Net,Open,2022-01-17,None,CAD,Los Angeles


In [3]:
import pandas as pd
import sqlite3

# Step 1: Load CSV into a DataFrame
csv_path = '/Users/vikalp.bhatnagar/Downloads/claim_investigation_data.csv'
df = pd.read_csv(csv_path)

# Optional: View the first few rows to ensure it's loaded correctly
df.head()
# Step 2: Connect to SQLite database
conn = sqlite3.connect('sampledata.db')  # if the file is in the current directory
cursor = conn.cursor()
# Step 3: Insert data into the 'loss_estimation' table
df.to_sql('ClaimInvestigation', conn, if_exists='append', index=False)
# Step 4: Verify the insert
pd.read_sql("SELECT * FROM ClaimInvestigation LIMIT 5;", conn)

,ClaimNumber,AdjusterID,InvestigationStatus,InvestigationCost
0,CLM004057,1288,Completed,4593.43
1,CLM000219,1631,In Review,159.93
2,CLM001105,1693,Completed,1478.49
3,CLM001209,1184,Pending,2293.29
4,CLM004549,1216,Completed,1663.55


In [9]:
import vanna as vn
from vanna.remote import VannaDefault
# from vanna.openai.openai_chat import OpenAI_Chat
#https://vanna.ai/account/login

# Step 1: Initialize model
vanna_api_key = 'f1278f1ebabf46d0ba68a4f7f87b9cd8'
vanna_model_name = 'pc-model'
vn = VannaDefault(model= vanna_model_name, api_key = vanna_api_key)

conn = sqlite3.connect('sampledata.db')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])




Tables in the database:
PNC_Claims
ClaimInvestigation


In [10]:
vn.train(ddl="""
CREATE TABLE PNC_Claims (
    ClaimNumber VARCHAR(30) NOT NULL PRIMARY KEY,
    Reserve_Profile VARCHAR(50),
    ExposureYear INT,
    IncurredLoss FLOAT,
    UltimateLoss FLOAT,
    AssignedEarmarkedLoss FLOAT,
    FinancePremium FLOAT,
    LossType VARCHAR(10),
    ClaimStatus VARCHAR(20),
    ReportedDate DATETIME,
    SettlementDate DATETIME,
    Currency VARCHAR(5),
    GeographicalLocation VARCHAR(50)
);
""")
vn.train(ddl="""
CREATE TABLE ClaimInvestigation (
    ClaimNumber VARCHAR(30) NOT NULL PRIMARY KEY,
    AdjusterID INT,
    InvestigationStatus VARCHAR(20),
    InvestigationCost FLOAT,
    FOREIGN KEY (ClaimNumber) REFERENCES PNC_Claims(ClaimNumber)
);
""")


Adding ddl: 
CREATE TABLE PNC_Claims (
    ClaimNumber VARCHAR(30) NOT NULL PRIMARY KEY,
    Reserve_Profile VARCHAR(50),
    ExposureYear INT,
    IncurredLoss FLOAT,
    UltimateLoss FLOAT,
    AssignedEarmarkedLoss FLOAT,
    FinancePremium FLOAT,
    LossType VARCHAR(10),
    ClaimStatus VARCHAR(20),
    ReportedDate DATETIME,
    SettlementDate DATETIME,
    Currency VARCHAR(5),
    GeographicalLocation VARCHAR(50)
);

Adding ddl: 
CREATE TABLE ClaimInvestigation (
    ClaimNumber VARCHAR(30) NOT NULL PRIMARY KEY,
    AdjusterID INT,
    InvestigationStatus VARCHAR(20),
    InvestigationCost FLOAT,
    FOREIGN KEY (ClaimNumber) REFERENCES PNC_Claims(ClaimNumber)
);



'9917948-ddl'

In [11]:
vn.train(documentation="""
PNC_Claims Table:
- LossType column contains 'Gross' for total claim value before deductions and 'Net' for the amount after recoveries or adjustments.
- ClaimStatus may be 'Open', 'Closed', or 'Reopened' based on the lifecycle of the claim.
- ReportedDate and SettlementDate are stored in ISO format (YYYY-MM-DD). SettlementDate may be NULL if the claim is still open.
- Currency field holds values like 'USD', 'EUR', 'GBP', etc., representing the currency in which the claim is processed.
- GeographicalLocation indicates the branch or region handling the claim (e.g., 'New York', 'London', 'Berlin').

ClaimInvestigation Table:
- InvestigationStatus may be 'Pending', 'In Review', or 'Completed', indicating the progress of the claim investigation.
- InvestigationCost is a floating-point value representing the total cost incurred during the investigation process.
- AdjusterID is a unique numeric identifier assigned to each insurance adjuster handling the claim.
""")


Adding documentation....


'3403187-doc'

In [13]:
vn.train(
    question="What is the total incurred loss for all claims reported in 2023?",
    sql="""
    SELECT SUM(IncurredLoss) AS TotalIncurredLoss
    FROM PNC_Claims
    WHERE strftime('%Y', ReportedDate) = '2023';
    """
)

vn.train(
    question="How many claims were reported in each geographical location?",
    sql="""
    SELECT GeographicalLocation, COUNT(*) AS NumberOfClaims
    FROM PNC_Claims
    GROUP BY GeographicalLocation;
    """
)

vn.train(
    question="List all open claims with their ultimate loss and investigation status.",
    sql="""
    SELECT p.ClaimNumber, p.UltimateLoss, i.InvestigationStatus
    FROM PNC_Claims p
    LEFT JOIN ClaimInvestigation i ON p.ClaimNumber = i.ClaimNumber
    WHERE p.ClaimStatus = 'Open';
    """
)


'bc5569ba3adb6348f196004299e48892-sql'

In [14]:
vn.train(
    question="What is the total incurred loss for all claims reported in 2023?",
    sql="""
    SELECT SUM(IncurredLoss) AS TotalIncurredLoss
    FROM PNC_Claims
    WHERE strftime('%Y', ReportedDate) = '2023';
    """
)

vn.train(
    question="How many claims were reported in each geographical location?",
    sql="""
    SELECT GeographicalLocation, COUNT(*) AS NumberOfClaims
    FROM PNC_Claims
    GROUP BY GeographicalLocation;
    """
)

vn.train(
    question="List all open claims with their ultimate loss and investigation status.",
    sql="""
    SELECT p.ClaimNumber, p.UltimateLoss, i.InvestigationStatus
    FROM PNC_Claims p
    LEFT JOIN ClaimInvestigation i ON p.ClaimNumber = i.ClaimNumber
    WHERE p.ClaimStatus = 'Open';
    """
)

vn.train(
    question="Which adjusters have the highest average investigation cost?",
    sql="""
    SELECT AdjusterID, AVG(InvestigationCost) AS AvgCost
    FROM ClaimInvestigation
    GROUP BY AdjusterID
    ORDER BY AvgCost DESC
    LIMIT 5;
    """
)

vn.train(
    question="Get the total number of claims and total investigation cost by status.",
    sql="""
    SELECT InvestigationStatus, COUNT(*) AS TotalClaims, SUM(InvestigationCost) AS TotalCost
    FROM ClaimInvestigation
    GROUP BY InvestigationStatus;
    """
)

vn.train(
    question="Find all claims where the ultimate loss is greater than 100,000 and reported in USD.",
    sql="""
    SELECT ClaimNumber, UltimateLoss, Currency
    FROM PNC_Claims
    WHERE UltimateLoss > 100000 AND Currency = 'USD';
    """
)

vn.train(
    question="What is the average time to settlement for closed claims?",
    sql="""
    SELECT AVG(julianday(SettlementDate) - julianday(ReportedDate)) AS AvgDaysToSettle
    FROM PNC_Claims
    WHERE ClaimStatus = 'Closed' AND SettlementDate IS NOT NULL;
    """
)

vn.train(
    question="List all claims with their adjuster and investigation status.",
    sql="""
    SELECT p.ClaimNumber, p.ClaimStatus, i.AdjusterID, i.InvestigationStatus
    FROM PNC_Claims p
    INNER JOIN ClaimInvestigation i ON p.ClaimNumber = i.ClaimNumber;
    """
)

vn.train(
    question="Find claims where the investigation cost exceeds the assigned earmarked loss.",
    sql="""
    SELECT p.ClaimNumber, p.AssignedEarmarkedLoss, i.InvestigationCost
    FROM PNC_Claims p
    INNER JOIN ClaimInvestigation i ON p.ClaimNumber = i.ClaimNumber
    WHERE i.InvestigationCost > p.AssignedEarmarkedLoss;
    """
)

vn.train(
    question="Rank claims by incurred loss within each geographical location.",
    sql="""
    SELECT ClaimNumber, GeographicalLocation, IncurredLoss,
           RANK() OVER (PARTITION BY GeographicalLocation ORDER BY IncurredLoss DESC) AS LossRank
    FROM PNC_Claims;
    """
)

vn.train(
    question="Get the number of high-value investigations (over 3000) by adjuster.",
    sql="""
    SELECT AdjusterID, COUNT(*) AS HighValueInvestigations
    FROM ClaimInvestigation
    WHERE InvestigationCost > 3000
    GROUP BY AdjusterID;
    """
)

vn.train(
    question="Which closed claims took more than 6 months to settle?",
    sql="""
    SELECT ClaimNumber, ReportedDate, SettlementDate,
           (julianday(SettlementDate) - julianday(ReportedDate)) / 30.0 AS MonthsToSettle
    FROM PNC_Claims
    WHERE ClaimStatus = 'Closed' AND SettlementDate IS NOT NULL
          AND (julianday(SettlementDate) - julianday(ReportedDate)) > 180;
    """
)

vn.train(
    question="Show cumulative incurred loss by exposure year.",
    sql="""
    SELECT ExposureYear, ClaimNumber, IncurredLoss,
           SUM(IncurredLoss) OVER (PARTITION BY ExposureYear ORDER BY ClaimNumber) AS CumulativeLoss
    FROM PNC_Claims;
    """
)

vn.train(
    question="List claims with above-average investigation cost.",
    sql="""
    SELECT i.ClaimNumber, i.InvestigationCost
    FROM ClaimInvestigation i
    WHERE i.InvestigationCost > (
        SELECT AVG(InvestigationCost) FROM ClaimInvestigation
    );
    """
)

'517c3768e4525e18251531751c9306fa-sql'

In [15]:
# Filter by month and year
vn.train(
    question="How many claims were reported in January 2024?",
    sql="""
    SELECT COUNT(*) AS ClaimsInJan2024
    FROM PNC_Claims
    WHERE strftime('%Y-%m', ReportedDate) = '2024-01';
    """
)

# Classify claims by severity using CASE
vn.train(
    question="Classify claims into severity bands based on incurred loss.",
    sql="""
    SELECT ClaimNumber, IncurredLoss,
           CASE
               WHEN IncurredLoss >= 100000 THEN 'High'
               WHEN IncurredLoss >= 50000 THEN 'Medium'
               ELSE 'Low'
           END AS SeverityBand
    FROM PNC_Claims;
    """
)

# Subquery with join
vn.train(
    question="Which claims have a higher investigation cost than the average for their status?",
    sql="""
    SELECT i.ClaimNumber, i.InvestigationStatus, i.InvestigationCost
    FROM ClaimInvestigation i
    WHERE i.InvestigationCost > (
        SELECT AVG(InvestigationCost)
        FROM ClaimInvestigation
        WHERE InvestigationStatus = i.InvestigationStatus
    );
    """
)

# Find most recent claims per location using window function
vn.train(
    question="Get the most recent claim reported in each geographical location.",
    sql="""
    SELECT ClaimNumber, GeographicalLocation, ReportedDate
    FROM (
        SELECT ClaimNumber, GeographicalLocation, ReportedDate,
               ROW_NUMBER() OVER (PARTITION BY GeographicalLocation ORDER BY ReportedDate DESC) AS rn
        FROM PNC_Claims
    )
    WHERE rn = 1;
    """
)

# Aggregate by claim status and month
vn.train(
    question="What is the monthly count of closed claims?",
    sql="""
    SELECT strftime('%Y-%m', SettlementDate) AS Month, COUNT(*) AS ClosedClaims
    FROM PNC_Claims
    WHERE ClaimStatus = 'Closed' AND SettlementDate IS NOT NULL
    GROUP BY Month
    ORDER BY Month;
    """
)

# Detect unusually high ultimate losses using threshold
vn.train(
    question="Which claims have unusually high ultimate losses over 90th percentile?",
    sql="""
    SELECT ClaimNumber, UltimateLoss
    FROM PNC_Claims
    WHERE UltimateLoss > (
        SELECT PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY UltimateLoss)
        FROM PNC_Claims
    );
    """
)

'198d0cfc2059be8e53c372aedef88180-sql'

In [16]:
vn.get_training_data()

,id,training_data_type,question,content
0,1129766-sql,sql,Get the number of high-value investigations (o...,"\n SELECT AdjusterID, COUNT(*) AS HighValue..."
1,1129759-sql,sql,Which adjusters have the highest average inves...,"\n SELECT AdjusterID, AVG(InvestigationCost..."
2,1129761-sql,sql,Find all claims where the ultimate loss is gre...,"\n SELECT ClaimNumber, UltimateLoss, Curren..."
3,1129768-sql,sql,Show cumulative incurred loss by exposure year.,"\n SELECT ExposureYear, ClaimNumber, Incurr..."
4,1129756-sql,sql,What is the total incurred loss for all claims...,\n SELECT SUM(IncurredLoss) AS TotalIncurre...
5,1129764-sql,sql,Find claims where the investigation cost excee...,"\n SELECT p.ClaimNumber, p.AssignedEarmarke..."
6,1129767-sql,sql,Which closed claims took more than 6 months to...,"\n SELECT ClaimNumber, ReportedDate, Settle..."
7,9917948-ddl,ddl,None,\nCREATE TABLE ClaimInvestigation (\n Claim...
8,1129770-sql,sql,How many claims were reported in January 2024?,\n SELECT COUNT(*) AS ClaimsInJan2024\n ...
9,1129758-sql,sql,List all open claims with their ultimate loss ...,"\n SELECT p.ClaimNumber, p.UltimateLoss, i...."
